In [4]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)
oracle_cur = connection.cursor()


In [6]:
oracle_cur.execute(oracle_test)

# Fetch the results using fetchall or fetchone, depending on your needs
test_results = oracle_cur.fetchall()

# Now test_results contains the results of your query
# print(test_results)
# test_results['seq']
test_results

[(21046710,
  242,
  '025158',
  'KB',
  ' ',
  'A',
  'B',
  ' ',
  '999 999 9999',
  'FAX NUMBER',
  750,
  ' ',
  'D',
  1,
  ' ',
  0,
  100000,
  ' ',
  'D',
  100000,
  ' ',
  'D',
  0,
  ' ',
  1.29,
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  95,
  ' ',
  'DICKINSON BRANDS -202',
  'NEED TO HAVE NEW ADDRESS',
  'XXXXXXXXXX',
  'NJ',
  '07013',
  '  ',
  'FRANK PARISE',
  'I',
  '13591',
  'C',
  '000000000',
  '    ',
  0,
  'N',
  67125,
  0,
  ' ',
  None,
  2,
  ' ',
  30,
  ' ',
  31,
  ' ',
  'G',
  'N',
  0,
  0,
  '    ',
  'V',
  0,
  ' ',
  None,
  0,
  ' ',
  None,
  'Y',
  'N',
  'N',
  'N',
  'Y',
  'N',
  0,
  ' ',
  None,
  0,
  None,
  None,
  9999999,
  ' ',
  0,
  ' ',
  None,
  None,
  'V120509185282000004018',
  None,
  'US',
  'Y',
  'W',
  '20',
  None,
  'P',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'N',
  374327,
  '25158',
  'CONTINENTAL POLY BA

In [7]:
try:
    # Step 1: Fetch data from Oracle table
#     oracle_test = "SELECT * FROM your_oracle_table_name"
    oracle_cur.execute(oracle_test)
    
    # Get Oracle column names
    oracle_columns = [desc[0] for desc in oracle_cur.description]

    # Map Oracle column names to PostgreSQL column names
    column_mapping = {
        'BIC_CREATION_DATE': 'CREATION_DATE',
        # Add more mappings as needed
    }

    # Modify Oracle column names based on the mapping
    modified_oracle_columns = [column_mapping.get(col, col) for col in oracle_columns]

    # Construct the INSERT INTO statement with dynamic column names
    insert_query = f"""
        INSERT INTO XXPO_ORCL2CSPOMS_VEND_STG (PROCESS_STATUS, {', '.join(modified_oracle_columns[1:])})
        VALUES (%s, {', '.join(['%s'] * (len(modified_oracle_columns)-1))})
    """

    # Fetch data from Oracle
    oracle_data = oracle_cur.fetchall()

    # Split the data based on the first column
    data_with_value = [row[1:] for row in oracle_data if row[0] is not None]

    # Insert data with values into PostgreSQL with 'U' for PROCESS_STATUS
    # Execute the INSERT query with the data
    post_cur.executemany(insert_query, [('U',) + row for row in data_with_value])

    # Get the number of inserted rows
    inserted_rows_count = post_cur.rowcount

    print(f"{inserted_rows_count} records inserted successfully.")


    # Commit the changes in PostgreSQL
    post_conn.commit()
    print('Records loaded to STG from Oracle Delta run')

except Exception as e:
    print(f"An error occurred: {e}")
# finally:
#     # Close Oracle and PostgreSQL connections and cursors
#     oracle_cur.close()
#     post_cur.close()
#     post_conn.close()


60 records inserted successfully.
Records loaded to STG from Oracle Delta run


In [9]:
import psycopg2

def delta_oracle_to_stg(oracle_cur, post_cur, oracle_test):
    try:
        # Get Oracle column names
        oracle_cur.execute(oracle_test)
        oracle_columns = [desc[0] for desc in oracle_cur.description]

        # Map Oracle column names to PostgreSQL column names
        column_mapping = {
            'BIC_CREATION_DATE': 'CREATION_DATE',
            # Add more mappings as needed
        }

        # Modify Oracle column names based on the mapping
        modified_oracle_columns = [column_mapping.get(col, col) for col in oracle_columns]

        # Construct the INSERT INTO statement with dynamic column names for U
        insert_query_u = f"""
            INSERT INTO XXPO_ORCL2CSPOMS_VEND_STG (PROCESS_STATUS, {', '.join(modified_oracle_columns[1:])})
            VALUES (%s, {', '.join(['%s'] * (len(modified_oracle_columns)-1))})
        """
        # Construct the INSERT INTO statement with dynamic column names for E
        insert_query_u = f"""
            INSERT INTO XXPO_ORCL2CSPOMS_VEND_STG (PROCESS_STATUS, {', '.join(modified_oracle_columns[1:])})
            VALUES (%s, {', '.join(['%s'] * (len(modified_oracle_columns)-1))})
        """

        # Fetch data from Oracle
        oracle_cur.execute(oracle_test)
        oracle_data = oracle_cur.fetchall()

        # Split the data based on the first column
        data_with_value = [row[1:] for row in oracle_data if row[0] is not None]

        # Insert data with values into PostgreSQL with 'U' for PROCESS_STATUS
        post_cur.executemany(insert_query_u, [('U',) + row for row in data_with_value])

        # Get the number of inserted rows
        inserted_rows_count = post_cur.rowcount

        print(f"{inserted_rows_count} records inserted successfully.")


        # Commit the changes in PostgreSQL
        post_conn.commit()
        print('Records loaded to STG from Oracle Delta run')
    except Exception as e:
        print(f"An error occurred: {e}")


delta_oracle_to_stg(oracle_cur, post_cur, oracle_test)

# # Close Oracle and PostgreSQL connections and cursors
# oracle_cur.close()
# oracle_conn.close()
# post_cur.close()
# post_conn.close()


60 records inserted successfully.
Records loaded to STG from Oracle Delta run


In [3]:
oracle_test = '''
WITH numbered_rows AS (
    SELECT
        vm.*,
        bic.*,
                bic.CREATION_DATE AS BIC_CREATION_DATE,

        ROW_NUMBER() OVER (PARTITION BY vm.vendor_home_loc_dest, bic.bic_vendor_facility, vm.vendor_nbr, bic.bic_vendor_number ORDER BY bic.stg_seq_id) AS rn
    FROM
        apps.xxap_vendor_master vm
        LEFT JOIN apps.xxpo_orcl2biceps_vend_stg bic 
        ON CASE WHEN LENGTH(vm.vendor_nbr) < 6 THEN LPAD(vm.vendor_nbr, 6, '0') ELSE vm.vendor_nbr END = bic.bic_vendor_number(+)
    WHERE
    bic.STG_SEQ_ID is not null
    --AND rownum < 100
    AND ROWNUM < 100
    --vm.LAST_UPDATE_DATE > SYSDATE - 1
    --vm.vendor_nbr IN ('8571', '445932', '447709', '10000044') 
) 
SELECT
STG_SEQ_ID                          
,BIC_VENDOR_FACILITY                 
,BIC_VENDOR_NUMBER                   
,BIC_BUYER_NUMBER                    
,BIC_CRP_LEVEL   
,BIC_STATUS                          
,BIC_PO_RECOMMENDED_IND              
,BIC_AFE_TYPE1                       
,BIC_AFE_FAX_NUMBER                  
,BIC_AFE_CONTACT                     
,BIC_MINIMUM_QUANTITY                
,BIC_MINIMUM_QUANTITY_SIGN           
,BIC_MINIMUM_TYPE                    
,BIC_BRACKET_QUANTITY                
,BIC_BRACKET_QUANTITY_SIGN           
,BIC_CURRENT_BKT_NUMBER              
,BIC_CURRENT_BKT_QUANTITY            
,BIC_CURRENT_BKT_QUANTITY_SIGN       
,BIC_CURRENT_BKT_TYPE                
,BIC_MAXIMUM_QUANTITY                
,BIC_MAXIMUM_QUANTITY_SIGN           
,BIC_MAXIMUM_TYPE                    
,BIC_ORDER_INTERVAL_WEEKS            
,BIC_ORDER_INTERVAL_WEEKS_SIGN       
,BIC_LEAD_TIME_STATED_WEEKS          
,BIC_LEAD_TIME_STATED_WKS_SIGN       
,BIC_FIXED_REVIEW_DAY1               
,BIC_FIXED_REVIEW_DAY2               
,BIC_FIXED_REVIEW_DAY3               
,BIC_FIXED_REVIEW_DAY4               
,BIC_FIXED_REVIEW_DAY5               
,BIC_FIXED_REVIEW_DAY6               
,BIC_FIXED_REVIEW_DAY7               
,BIC_TARGET_SERVICE_LEVEL            
,BIC_TARGET_SERVICE_LEVEL_SIGN       
,BIC_NAME                            
,BIC_ADDRESS_1                       
,BIC_CITY                            
,BIC_STATE                           
,BIC_ZIP                             
,BIC_PHONE                           
,BIC_CONTACT                         
,BIC_TRANSFER_MATCH_TYPE             
,BIC_PAYABLE_VENDOR_NBR              
,BIC_LEAD_TIME_WEEKS_TYPE            
,BIC_DUNS_REMIT_NO                   
,BIC_DUNS_REMIT_SUFFIX               
,BIC_FACILITY_SHIP_TO                
,BIC_PRE_SCHED_FLAG                  
,BIC_BROKER_NUMBER                   
,BIC_FREIGHT_ALLOW                   
,BIC_FREIGHT_ALLOW_SIGN              
,BIC_FREIGHT_ALLOW_TYPE              
,BIC_TERMS_PERCENT                   
,BIC_TERMS_PERCENT_SIGN              
,BIC_TERMS_DAYS                      
,BIC_TERMS_DAYS_SIGN                 
,BIC_TERMS_NET_DAYS                  
,BIC_TERMS_NET_DAYS_SIGN             
,BIC_TERMS_BASE                      
,BIC_BILL_FOR_SHORT_DEALS_FLAG       
,BIC_TRANSIT_DAYS                    
,BIC_DUNS_NO                         
,BIC_DUNS_SUFFIX                     
,BIC_VENDOR_TYPE                     
,BIC_2ND_MIN_QUANTITY                
,BIC_2ND_MIN_QUANTITY_SIGN           
,BIC_2ND_MIN_TYPE                    
,BIC_2ND_MAX_QUANTITY                
,BIC_2ND_MAX_QUANTITY_SIGN           
,BIC_2ND_MAX_TYPE                    
,BIC_FLAG_PREPAID                    
,BIC_FLAG_PREPAY_AND_ADD             
,BIC_FLAG_FREIGHT_BILL               
,BIC_FLAG_BACKHAUL                   
,BIC_FLAG_TRUCK                      
,BIC_FLAG_RAIL                       
,BIC_BACKHAUL                        
,BIC_BACKHAUL_SIGN                   
,BIC_BACKHAUL_TYPE                   
,BIC_FREIGHTBILL                     
,BIC_FREIGHTBILL_SIGN                
,BIC_FREIGHTBILL_TYPE                
,BIC_ORDER_SEQUENCE                  
,BIC_ORDER_SEQUENCE_SIGN             
,BIC_SSA_ALLOW_PERCENT               
,BIC_SSA_ALLOW_PERCENT_SIGN          
,BIC_SSA_ALLOW_BASIS                 
,BIC_SSA_ALLOW_ACCOUNT               
,BIC_TRANSACTION_ID                  
,BIC_SSA_FLAG                        
,BIC_COUNTRY                         
,BIC_FILTER_VENDOR_FLAG              
,BIC_LOAD_BLDG_LIMIT_FLAG            
,BIC_WAREHOUSE_CODE                  
,BROKER_PROCESS_STATUS               
,BV_PROCESS_STATUS       
,ERROR_MESSAGE

                  
,ATTRIBUTE_CATEGORY                  
,ATTRIBUTE1                          
,ATTRIBUTE2                          
,ATTRIBUTE3                          
,ATTRIBUTE4                          
,ATTRIBUTE5                          
,ATTRIBUTE6                          
,ATTRIBUTE7                          
,ATTRIBUTE8                          
,ATTRIBUTE9                          
,ATTRIBUTE10                         
,ATTRIBUTE11                         
,ATTRIBUTE12                         
,ATTRIBUTE13                         
,ATTRIBUTE14                         
,ATTRIBUTE15                         
,BIC_EXTERNAL_BACKHAUL
,VENDOR_SITE_ID                  
,VENDOR_NBR                      
,VENDOR_NAME                     
,VENDOR_START_DATE               
,VENDOR_END_DATE                 
,VENDOR_STATUS                   
,VENDOR_TYPE                     
,AP_VENDOR_ID                    
,AP_VENDOR_NAME                  
,AP_VENDOR_ALT_NAME              
,AP_VENDOR_NBR                   
,AP_VENDOR_START_DATE            
,AP_VENDOR_END_DATE              
,AP_VENDOR_STATUS                
,COMPANY                         
,SUPPLIER_TYPE                   
,CONNECTING_VENDOR               
,BUYER_VENDOR                    
,BUYER_VENDOR_NAME               
,PAY_SITE_FLAG                   
,PURCHASING_SITE_FLAG            
,PRIMARY_PAY_SITE_FLAG           
,WHSE_VENDOR_NBR                 
,VENDOR_HOME_LOC_DEST            
,SHIP_TO_LOCATION_NAME           
,BILL_TO_LOCATION_NAME           
,VENDOR_SERVICE_LEVEL            
,VENDOR_MULTIPLE_ORDER           
,VENDOR_IMPLUS_FLG               
,VENDOR_TRANSPORT_METHOD_CODE    
,VENDOR_CONTINUITY_REPLEN_FLG    
,VENDOR_EMERG_LEAD_TIME_DAYS     
,VENDOR_INTRANET_BASED_BUY_FLG   
,VENDOR_DISCON_FLG               
,VENDOR_PAY_TERM                 
,VENDOR_CASH_DISC                
,VENDOR_DISC_PCT                 
,VENDOR_DISCOUNT_DAYS_NBR        
,VENDOR_NET_DAYS                 
,VENDOR_EDI_CUTOFF_TIME          
,PREFERRED_VENDOR_IND            
,ES3_VENDOR_FLG                  
,MDSE_BUYER_NBR                  
,VENDOR_FOODLINK_FLG             
,VENDOR_FREIGHT_FLG              
,LEAD_TIME_CALENDAR_IND          
,VENDOR_LEAD_DAYS                
,VENDOR_LEAD_TIME                
,VENDOR_EMRG_LEAD_TIME_DAYS      
,VENDOR_REAL_LEAD_TIME           
,VENDOR_ROUNDING_FLG             
,VENDOR_DLVR_SUNDAY_FLG          
,VENDOR_DLVR_MONDAY_FLG          
,VENDOR_DLVR_TUESDAY_FLG         
,VENDOR_DLVR_WEDNESDAY_FLG       
,VENDOR_DLVR_THURSDAY_FLG        
,VENDOR_DLVR_FRIDAY_FLG          
,VENDOR_DLVR_SATURDAY_FLG        
,VENDOR_ORDER_UNIT               
,VENDOR_MIN_ORDER_SIZE           
,VENDOR_ALT_MIN_ORDER_SIZE       
,VENDOR_MAX_ORDER_SIZE           
,VENDOR_ORDER2_UNIT              
,VENDOR_MIN_ORDER2_SIZE          
,VENDOR_MAX_ORDER2_SIZE          
,VENDOR_ORDER3_UNIT              
,VENDOR_MIN_ORDER3_SIZE          
,VENDOR_MAX_ORDER3_SIZE          
,VENDOR_BROKER_NBR               
,VENDOR_BROKER_NAME              
,VENDOR_CROSSROADS_GROUP         
,PERISHABLE_BROKER_INFO          
,VENDOR_CONTACT_NAME             
,ADDR_LINE1                      
,ADDR_LINE2                      
,ADDR_LINE3                      
,ADDR_LINE4                      
,CITY                            
,STATE                           
,ZIP_CODE                        
,COUNTY                          
,PROVINCE                        
,COUNTRY                         
,AREA_CODE                       
,PHONE_NBR                       
,FAX_AREA_CODE                   
,FAX_NBR                         
,EMAIL_ADDR                      
,VENDOR_BUYER_NBR                
,BICEPS_BUYER_CODE               
,BUYER_NAME
               
,CONNECTING_VENDOR_NAME          
,MDSE_BUYER_NAME                 
,SSA_BASIS                       
,LEGACY_VENDOR_NUM               
,BV_DISCONT_STATUS               
,EDI_COMPLIANCE_FEE              
,VENDOR_ONE_SOURCE_FLG           
,VENDOR_MEMO_FIELD               
,DVC_VENDOR_FLG                  
,LOST_INCOME_POTENTIAL           
,VENDOR_CROSSROADS_FLG           
,VENDOR_CROSSROADS_DATE          
,VENDOR_TOLRNCE                  
,VENDOR_PROGRAM_FLG              
,DIV_VNDR_FLG                    
,FAX_NUM_EXTN                    
,VENDOR_WORK_PHONE_NO            
,VENDOR_WORK_PHONE_EXTN          
,VENDOR_CELL_NO                  
,VENDOR_CELL_EXTN                
,CLAIM_BACKUP_TYPE               
,SSVC                            
,PROC_VENDOR_TYPE                
,VENDOR_FLG2                     
,FILTER_VENDOR_FLG               
,IMPLUS_TARGET_LVL               
,DAYS_TO_BUY_OVERRIDE            
,VENDOR_SUB_FLG                  
,VENDOR_MAX_ALLOW_SUPPLY         
,VENDOR_SUBCLASS                 
,FULL_PALLET_ORDERING_ONLY       
,DUCS_CONVERSION_CODE            
,VENDOR_FREE_ON_BOARD_POINT      
,VENDOR_STD_ORD_INT_NBR          
,AD_FEATH_2_PARM_FLG             
,LOAD_BLDG_LIMIT_FLG             
,D2S_VENDOR_FLG                  
,VENDOR_ARE_DIFF_DESTS_OK        
,VENDOR_MIL_SOLD_TO_ACCT_NBR     
,VENDOR_RDS_PRE_SCHLED_FLG       
,FM_VENDOR_NBR                   
,LOAD_TYPE                       
,ALL_ORDER                       
,VENDOR_LOAD_TIME_DAYS           
,VENDOR_ORIG_CONTACT_INFO_ID     
,VENDOR_ZONE                     
,REDUCE_RETAIL_EXT_FLG           
,UCS875_MCASE_COST               
,VENDOR_BD_COUNTER               
,VENDOR_INACTIVATE_USER          
,VENDOR_DISCONT_DATE             
,PARENT_VENDOR                   
,BACKHAUL_DAYS_TRANSIT           
,BACKHAUL_PICKUP_POINT           
,BACKHUAL_ADDRESS                
,BACKHAUL_CITY                   
,BACKHAUL_STATE                  
,BACKHAUL_TELEPHONE_NBR          
,BACKHAUL_EXTN                   
,BACKHAUL_HRS                    
,BD_FACTOR                       
,SPECIAL_OFF_INVOICE             
,EDI_PARTNER_ID                  
,EDI_VERSION                     
,VENDOR_DUNS_NBR                 
,UCS880_TRADING_PARTNER          
,UCS852_FLG                      
,UCS852_OCCURRENCE_FLG           
,UCS852_TRADING_PARTNER          
,MIL_DUNSNBR_FLG                 
,DUCS_BROKER_ID                  
,ES3_VENDOR_DUNS_NBR             
,VENDOR_ALLOW_TRANS              
,EDI852_PARTNER_NAME             
,VENDOR_DEST_NBR                 
,DEFAULT_DUNS_NBR                
,VMI_852_855_VERSION             
,DUCS_ACTIVE_FLG                 
,UCS867_FLG                      
,UCS852_GIS_FLG                  
,EDI_DEFAULT_DEST                
,DUCS_850_PO_FLG                 
,VENDOR_3PL_FLG                  
,VENDOR_3PL_CHAIN_NBR            
,VENDOR_3PL_CUST_VENDOR_NBR      
,GROUP_OWNER                     
,GL_CODE                         
,MANF_UPC                        
,VENDOR_DEFAULT_DEST_MER         
,VENDOR_CONSLTD_FLG              
,VNDR_COMM_MTHD                  
,VNDR_COMM_EMAIL                
,VNDR_COMM_FAX                   
,PWMW_BV_NAME                    
,PWMW_AP_NUM                     
,PWMW_AP_NAME                    
,PWMW_WHS_NUM                    
,PWMW_ITEM_DEPT                  
,VENDOR_FACILITY                 
,BICEPS_PO_TYPE                  
,ELIGIBLE_FOR_EMPTY_PO           
,OPPRTSTIC_DPS                   
,BUILD_TO_DAYS_SUPPLY            
,DAYS_TO_BUILD_TO                
,USE_CYCLE_STOCK_AS_MIN          
,TRANSIT_DAYS                    
,LAG_DAYS                        
,BICEPS_VENDOR_TYPE              
,ZERO_SAFETY_STOCK               
,USE_FAC_WHSE_SS                 
,APPLY_RATIONING                 
,DEMAND_DAYS                     
,OP_DAYS_SHIPPER                 
,OP_DAYS_1ST_RUN_1               
,OP_DAYS_1ST_RUN_2               
,OP_DAYS_1ST_RUN_3               
,OP_DAYS_1ST_RUN_4               
,OP_DAYS_1ST_RUN_5               
,OP_DAYS_1ST_RUN_6               
,OP_DAYS_1ST_RUN_7               
,OP_DAYS_2ND_RUN_1               
,OP_DAYS_2ND_RUN_2               
,OP_DAYS_2ND_RUN_3               
,OP_DAYS_2ND_RUN_4               
,OP_DAYS_2ND_RUN_5               
,OP_DAYS_2ND_RUN_6               
,OP_DAYS_2ND_RUN_7               
,MAIN_FACILITY                   
,LEAD_TIME_OVRD_DAYS_1           
,LEAD_TIME_OVRD_DAYS_2           
,LEAD_TIME_OVRD_DAYS_3           
,LEAD_TIME_OVRD_DAYS_4           
,LEAD_TIME_OVRD_DAYS_5           
,LEAD_TIME_OVRD_DAYS_6           
,LEAD_TIME_OVRD_DAYS_7           
,SIMULTD_BUYING_VENDOR           
,SIMULTD_BUYING_SPLIT            
,SIMULTD_APPR_SAT                
,SIMULTD_APPR_SUN                
,SIMULTD_APPR_HOLIDAY            
,COLBRTV_ORD_BLDG                
,SWELL_ALLOW_PCT                 
,SWELL_ALLOW_BASIS               
,SWELL_ALLOW_ACCT                
,PICK_SLOT_CPTY_ELIGIBLE         
,LOGSTCL_PROGRAM_FLAG            
,LOGSTCL_PROGRAM_PCT             
,PICKUP_DAY_1                    
,PICKUP_DAY_2                    
,PICKUP_DAY_3                    
,PICKUP_DAY_4                    
,PICKUP_DAY_5                    
,PICKUP_DAY_6                    
,PICKUP_DAY_7                    
,EOQ_CAP_DAYS                    
,WHSE_SHIP_TO                    
,SHIP_WITH_VENDOR                
,RANK_PERCENT_OVERRIDE           
,PO_DATE_CHANGE_RSN_FLAG         
,PULLBKS_AVAILABLE                            
,BIC_CREATION_DATE             
      
FROM
    numbered_rows
WHERE
    rn = 1


'''

In [10]:
print(2)

2
